# Differential (Predictive) Coding

## Motivation

* Usually happens that the differences between consecutive PCM (Pulse-Code Modulation) [[A.V. Oppenheim, 1999](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=oppenheim+discrete+time+signal+processing&btnG=&oq=Oppenheim)] samples

  $$
    e[n] = s[n] - s[n-1],
  $$

  tend to have a smaller variance (and therefore, smaller entropy) than the original ($s$) ones

  $$
    H(e) \leq H(s),
  $$

  which potentially provides better lossless compression ratios for $e$ than for $s$.

* Finally, notice that by definition, if a source of "noise" (such as the quantization noise) has not been introduced during the encoding process, differential encoding is a fully reversible process:

  $$
    s[n] = e[n] + s[n-1].
  $$

# Lab

* Compute $e$ signal for the [Jfk_berlin_address_high.ogg](https://upload.wikimedia.org/wikipedia/commons/3/3a/Jfk_berlin_address_high.ogg) $s$ signal and plot the probability density function (also known as the histogram) of $e$. Note: use Python and Matplotlib, and insert the code here.

## DPCM (Differential PCM) [[Cutler, 1952]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=%22Differential+quantization+of+communication+signals%22&btnG=)

* Signals sampled at Nyquist rate exhibit correlation between consecutive samples. Therefore, the variance of the first difference

  \begin{equation}
    \text{Var}(\{s[n+1]-s[n]\}_{n=0}^{N-1})
  \end{equation}
  
  will be smaller than the variance of the signal itself

  \begin{equation}
    \text{Var}(\{s[n+1]-s[n]\}_{n=0}^{N-1}) < \text{Var}(s) = \sigma_s^2.
  \end{equation}

* In general, DPCM consist in computing the residual (error signal):

  \begin{equation}
    e[n] = s[n] - \hat{s}[n]
  \end{equation}
  
  where for the simplest case:
  
  \begin{equation}
    \hat{s}[n] = s[n-1]
  \end{equation}

  <img src="figs/DPCM.png" width="400">

* Used in the [G.726 standard](https://en.wikipedia.org/wiki/G.726).

## ADPCM (Adaptive DPCM)

* The predictor can change its behaviour depending on the characteristics of $s$.

## Forward adaptive prediction

* Splits the input into blocks, optimizes the predictor for each block and sends in each block que predictor coefficients as side information.

## Optimal prediction based on Weiner-Hopf optimization

* The predictor can be any structure capable of producing a signal

  \begin{equation}
    \hat{s} \approx s.
  \end{equation}

* In the case of using LPC (Linear Predictive Coding), where

  \begin{equation}
    \hat{s}[n] = \sum_{i=1}^M a_i s[n-i],
  \end{equation}
  
  $\{a_i\}$ are the LPC coefficients and $M$ is the predictor order.
  
* Coeffs $\{a_i\}$ can be found when they minimize the variance of the residue (error signal)
  
  \begin{equation}
    \delta_e^2 = \text{E}\big[(s[n]-\sum_{i=1}^M a_i s[n-i])^2\big].
  \end{equation}
  
* For this, it must me hold that

  \begin{equation}
    \left.
      \begin{array}{c}
        \frac{\text{d}\delta_e^2}{\text{d}a_1} = -2\text{E}\Big[\big(s[n]-\displaystyle\sum_{i=1}^M a_i s[n-i]\big)s[n-1]\Big] = 0 \\
        \frac{\text{d}\delta_e^2}{\text{d}a_2} = -2\text{E}\Big[\big(s[n]-\displaystyle\sum_{i=1}^M a_i s[n-i]\big)s[n-12\Big] = 0 \\
        \vdots \\
        \frac{\text{d}\delta_e^2}{\text{d}a_M} = -2\text{E}\Big[\big(s[n]-\displaystyle\sum_{i=1}^M a_i s[n-i]\big)s[n-M]\Big] = 0 
      \end{array}
    \right\rbrace.
  \end{equation}
  
* Appying expectations

  \begin{equation}
    \left.
      \begin{array}{c}
        \displaystyle\sum_{i=1}^M a_i \text{R}_{ss}(i-1) = \text{R}_{ss}(1) \\
        \displaystyle\sum_{i=1}^M a_i \text{R}_{ss}(i-2) = \text{R}_{ss}(2) \\
        \vdots\\
        \displaystyle\sum_{i=1}^M a_i \text{R}_{ss}(i-N) = \text{R}_{ss}() \\
      \end{array}
    \right\rbrace,
    \tag{Weiner-Hopf equations}
    \label{Weiner-Hopf equations}
  \end{equation}
  
  where $\text{R}_{ss}(k)$ is the autorrelation function of $s$, defined as:
  
  \begin{equation}
    \text{R}_{ss}(k) = \text{E}\big[s[n]s[n-k]\big] = \frac{1}{M-k}\sum_{i=1}^{M-k} s[i]s[i+k].
    \tag{autocorrelation}
    \label{autocorrelation}
  \end{equation}

## Lab
For each $M=\{1,2,3\}$, determine the optimal predictor for the audio [Jfk_berlin_address_high.ogg](https://upload.wikimedia.org/wikipedia/commons/3/3a/Jfk_berlin_address_high.ogg). Next, compute the variance of $e$ for each prediction order.

## Backward adaptive prediction

* Both, the encoder and the decoder can constinuosly adapt the prediction to the characteristics of $s$, using the information that is available at the decoder.

## Adaptive prediction based on Least Mean Squared (LMS) algorithm

* As we know,

  \begin{equation}
    e[n] = s[n] = \hat{s}[n] = s[n] - \sum_{i=1}^M a_is[n-i].
  \end{equation}
  
* Our objective is to minimize $e[n]$, which is the same that minimizing the energy of the prediction error $e^2$, by controlling the $\{a_i\}_{i=1}^M$ coefficients.

* Suppose that we can control iteratively these coefficients, by incrementing or decrementing them with a proportionality constant $\alpha$ (the larger $\alpha$, the faster the convergence and viceversa). Let's denote $a_i^{[n]}$ the value of coeff $a_i$ at iteration $n$ of the algorithm. If happens, for example, that

  \begin{equation}
    \frac{\text{d}e^2[n]}{\text{d}a_i} < 0
  \end{equation}
  
  (the squared prediction error for iteration $n$ has been increased), then the prediction $\hat{s}[n]$ has been too small, and viceversa. Let's define the iterative control of $\{a_i\}_{i=1}^M$ coefficients as
  
  \begin{equation}
    \Big\{a_i^{[n-1]} = a_i^{[n]} - \alpha\frac{\text{d}e^2[n]}{\text{d}a_i}\Big\}_{i=1}^M
    \tag{LMS_coeffs_control}
  \end{equation}
  
* Applyig derivatives, for example, for $a_i$ we obtain that, 

  \begin{equation}
    \frac{\text{d}e^2[n]}{\text{d}a_i} = \frac{\text{d}\Big(s[n] - \sum_{i=1}^M a_is[n-i]\Big)^2}{\text{d}a_i} = -2\Big(s[n] - \sum_{i=1}^M a_is[n-i]\Big)^2\Big)s[n-i] = -2e[n]s[n-i].
    \end{equation}
    
* Substituting this expression in Eqs. LMS_coeffs_control, we get that

  \begin{equation}
    \Big\{a_i^{[n-1]} = a_i^{[n]} +2\alpha e[n]s[n-i]\Big\}_{i=1}^M.
    \tag{LMS_coeffs_control_final}
  \end{equation}

* Notice that, in a backward adaptive prediction scheme, both $e[n]$ and $s[n-1]$ are known at iteration $n$.

## Lossy DPCM [[Cutler, 1952]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=%22Differential+quantization+of+communication+signals%22&btnG=)

<img src="figs/QDPCM.png\" width=800>

* The prediction error is quantized to decrease the variance:

  \begin{equation}
    \tilde{e}[n] = Q(e[n])
  \end{equation}
  
  where
  
  \begin{equation}
    e[n] = s[n] - \hat{\tilde{s}}[n]
  \end{equation}
  
  where
  
  \begin{equation}
    \tilde{s}[n] = \hat{\tilde{s}}[n] + \tilde{e}[n].
  \end{equation}

* Used in [hybrid (lossy DPCM/transform) video coding](https://en.wikipedia.org/wiki/High_Efficiency_Video_Coding#Video_coding_layer).